In [8]:
import cv2
import typing
import numpy as np

from mltu.inferenceModel import OnnxInferenceModel
from mltu.utils.text_utils import ctc_decoder, get_cer
import pandas as pd
from tqdm import tqdm
from mltu.configs import BaseModelConfigs
import matplotlib.pyplot as plt

In [2]:

class ImageToWordModel(OnnxInferenceModel):
    def __init__(self, char_list: typing.Union[str, list], *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.char_list = char_list

    def predict(self, image: np.ndarray):
        image = cv2.resize(image, self.input_shape[:2][::-1])

        image_pred = np.expand_dims(image, axis=0).astype(np.float32)

        preds = self.model.run(None, {self.input_name: image_pred})[0]

        text = ctc_decoder(preds, self.char_list)[0]

        return text

In [ ]:
configs = BaseModelConfigs.load("/home/tuandinh/Desktop/Captra_projects/Models/02_captcha_to_text/202310071259/configs.yaml")

model = ImageToWordModel(model_path=configs.model_path, char_list=configs.vocab)

df = pd.read_csv("/home/tuandinh/Desktop/Captra_projects/Models/02_captcha_to_text/202310071259/val.csv").values.tolist()

accum_cer = []
result_prediction = []

for image_path, label in tqdm(df):
    image = cv2.imread(image_path)

    prediction_text = model.predict(image)
    result_prediction.append(prediction_text)
    cer = get_cer(prediction_text, label)
    print(f"Image: {image_path}, Label: {label}, Prediction: {prediction_text}, CER: {cer}")

    accum_cer.append(cer)

print(f"Average CER: {np.average(accum_cer)}")

In [5]:
result_prediction[1]

'3n3cf'

In [31]:
data = pd.read_csv("/home/tuandinh/Desktop/Captra_projects/Models/02_captcha_to_text/202310071259/val.csv")
data.shape
data_visulization = data.head(16)

16

,0,1
1,/home/tuandinh/Desktop/Captra_projects/Dataset...,3n3cf
2,/home/tuandinh/Desktop/Captra_projects/Dataset...,m4fd8
3,/home/tuandinh/Desktop/Captra_projects/Dataset...,bdbb3


0
0
1
1
